<a href="https://colab.research.google.com/github/diptiranjan1998/Raksha-pipes-assignment/blob/main/Diptiranjan_Bal_Raksha_Pipes_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing required libraries
import numpy as np
import pandas as pd
!pip install faker
from faker import Faker
import random


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### **1.Creating Customer Table**

In [ ]:
# Creating a function to replace ',' and '\'
def replace_comma(strings):
  output= strings.replace(',','-')
  output= output.replace("\n", "-")
  return output

In [ ]:
# Create an instance of the Faker class
fake = Faker()
Faker.seed(42)

# Generate fake data for 500 records
data = []
for _ in range(500):
    data.append({
        'customer_id':_,
        'customer_name': fake.name(),
        'customer_address': fake.address(),
        'customer_phone_number': fake.phone_number(),
        'birthdate': fake.date_between(start_date=pd.to_datetime('1990-01-01'), end_date=pd.to_datetime('2000-01-01'))
    })

# Create a DataFrame from the generated data
customer_info = pd.DataFrame(data)

# Creating fake relavant email
random.seed(42)
customer_info['customer_email']= customer_info['customer_name'].apply(lambda x:((''.join(x.split(' ')).lower())+str(random.choices(range(10,99),k=1)[0])+'@gmail.com'))

# Rearranging columns
customer_info = customer_info[['customer_id', 'customer_name', 'customer_address', 'customer_email', 'customer_phone_number', 'birthdate']]

# Replace ',' and '\'
customer_info['customer_address'] = customer_info['customer_address'].apply(replace_comma)

# Print the first few rows of the DataFrame
customer_info.head()

### **2.Creating Products Table**

In [ ]:
# Getting product names from company site
import requests
from bs4 import BeautifulSoup

# URL of the web page you want to scrape
url = 'https://www.rakshapipes.com/all-products.html'

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Extract data from the HTML
# For example, let's extract all the links (anchor tags) from the page
links = soup.find_all('a')
products= []
# Print the links
for link in links:
    # print(link.get('href'))
    if link.get('href')[:-5] not in products:
      products.append(link.get('href')[:-5])
products= products[1:]
products

In [ ]:
# Creating a function to get product types from product name
def extract_product_type(inp):
  inp_list= inp.split('-')
  if len(inp_list)==1:
    return inp_list[0]
  elif inp_list[0] != 'raksha':
    return inp_list[0]
  else:
    return inp_list[1]

In [ ]:
# Creating dataframe for products
data = []
random.seed(42)
random_product_names = random.choices(products, k=len(products))

for _ in range(len(products)):
    data.append({
        'product_id':_,
    })

# Create a DataFrame from the generated data
product_info = pd.DataFrame(data)
product_info['product_name']= random_product_names
product_info['product_type']= product_info['product_name'].apply(extract_product_type)

# Print the first few rows of the DataFrame
product_info.head()

### **3.Creating Order *info* Table**

In [ ]:
# Taking all the customer ids
customer_ids= customer_info['customer_id'].values.tolist()

In [ ]:
# Choosing random customer ids
random.seed(42)
random_customer_ids = [random.choice(customer_ids) for _ in range(200)]

# Creating a fake date for oprder_info dataframe
data = []
for _ in range(200):
  data.append({
      'order_id':_,
      'order_date':fake.date_between(start_date=pd.to_datetime('2024-01-01'), end_date=pd.to_datetime('2024-04-30'))})

# Creating the dataframe from fake data
order_info = pd.DataFrame(data)

# Adding random customer id
order_info['customer_id']= random_customer_ids

# Rearranging the tanble
order_info= order_info[['order_id','customer_id','order_date']]

# First look
order_info.head()

### **4.Creating Order *details* Table**

In [ ]:
# Taking all the order ids
order_ids = order_info['order_id'].values.tolist()

In [ ]:
# Taking all the product ids
product_ids = product_info['product_id'].values.tolist()

In [ ]:
# Choosing random order ids
random.seed(42)
random_order_ids = random.choices(order_ids, k= 200)

# Choosing random product ids
random.seed(42)
random_product_ids = [random.choice(product_ids) for _ in range(200)]

# Creating fake quantity
fake_quantity = range(20,50)
random.seed(42)
random_fake_quantity= [random.choice(fake_quantity) for _ in range(200)]

# Creating fake price
fake_price = range(20,50)
random_fake_price = [random.choice(fake_price) for _ in range(200)]

# Creating a fake date for oprder_info dataframe
data = []
for _ in range(200):
  data.append({
      'order_details_id':_})

# Creating a dataframe out of fake data
order_details= pd.DataFrame(data)

# Adding random order id
order_details['order_id'] = random_order_ids

# Adding random price. Here the price is in U.S Dollar
order_details['price_per_unit_in_USD'] = random_fake_price

# Adding random product id
order_details['product_id'] = random_product_ids

# Adding random quantity
order_details['quantity'] = random_fake_quantity

# Rearranging the tanble
order_details= order_details[['order_details_id', 'order_id', 'product_id', 'quantity', 'price_per_unit_in_USD']]

# First look
order_details.head()

### **5.Creating payment Table**


In [ ]:
# Taking order ids for payments
order_details_ids = order_details['order_details_id'].values.tolist()

# Taking only 150 orders, so that it looks realistic
random.seed(42)
payments_order_details_ids = order_details_ids[:150]
random.shuffle(payments_order_details_ids)

print(payments_order_details_ids)

In [ ]:
# Creating fake data for payments
data= []
for _ in range(150):
  data.append({'payment_id':_})

# Creating fake payments data frame
payments_info =  pd.DataFrame(data)

# Adding order details id
payments_info['order_details_id'] = payments_order_details_ids

# First look
payments_info.head()

In [ ]:
# Building relation between payments_info, order_details and order_info to get all payment details
payment_details = pd.merge(payments_info, order_details, how= 'inner', on= 'order_details_id')

payment_details = pd.merge(payment_details, order_info, how= 'inner', on= 'order_id')

# Getting total price payed
payment_details['total_price_payed_in_USD'] = payment_details['quantity'] * payment_details['price_per_unit_in_USD']

# First look
payment_details.head()

### **6.Creating inventory Table**


In [ ]:
# Creating fake availability
fake_availablity = list(range(50,500))
random.seed(42)
random_fake_availablity = [random.choice(fake_availablity) for _ in range(len(products))]

In [ ]:
# Creating a fake inventory table
inventory_info = pd.DataFrame()

# Adding product_id, product_name
inventory_info = product_info[['product_id','product_name']]

# Adding fake availability
inventory_info['availablity'] = random_fake_availablity

# First look
inventory_info.head()

### **7.Creating suppliers Table**


In [ ]:
# Creating fake availability
product_types= product_info['product_type'].values.tolist()
random.seed(42)
random_product_types = [random.choice(product_types) for _ in range(100)]


In [ ]:
# Create an instance of the Faker class
fake = Faker()
Faker.seed(42)

# Generate fake data for 100 records
data = []
for _ in range(100):
    data.append({
        'supplier_id':_,
        'supplier_name': fake.company(),
        'address': fake.address(),
        'email': fake.email(),
        'supplier_phone_number': fake.phone_number(),
    })

# Create a DataFrame from the generated data
supplier_info = pd.DataFrame(data)

# Adding product type for sopplier
supplier_info['product_type'] = random_product_types

# Print the first few rows of the DataFrame
supplier_info.head()

### **8.Creating employee Table**


In [ ]:
# Taking random post/position of the employee in the company
posts= ['Data scientist', 'Data analyst', 'Business analyst', 'Junior manager', 'Project head', 'Associate data scientist', 'Associate data analyst', 'MIS ececutive', 'Data entry', 'Business manager', 'Quality control', 'Production executive', 'Senior engineer', 'Operator']

# Creating random positions
random.seed(42)
fake_position = random.choices(posts, k= 100)

In [ ]:
salary_details = {'Data scientist':50000, 'Data analyst':40000, 'Business analyst':40000, 'Junior manager':45000, 'Project head':60000, 'Associate data scientist':40000, 'Associate data analyst':30000, 'MIS ececutive':25000, 'Data entry':20000, 'Business manager':45000, 'Quality control':30000, 'Production executive':25000, 'Senior engineer':28000, 'Operator':18000}

In [ ]:
# Create an instance of the Faker class
fake = Faker()
Faker.seed(42)

# Generate fake data for 100 records
data = []
for _ in range(100):
    data.append({
        'employee_id':_,
        'employee_name': fake.name(),
        'employee_address': fake.address(),
        'employee_phone_number': fake.phone_number(),
    })

# Creating fake data for employee
employee_info= pd.DataFrame(data)

employee_info['employee_position'] = fake_position

# Creating fake relavant email
random.seed(42)
employee_info['employee_email']= employee_info['employee_name'].apply(lambda x:((''.join(x.split(' ')).lower())+str(random.choices(range(10,99),k=1)[0])+'@gmail.com'))

# Adding salary column
employee_info['employee_salary'] = employee_info['employee_position'].apply(lambda x:salary_details[x])

# Replace ',' and '\'
employee_info['employee_address'] = employee_info['employee_address'].apply(replace_comma)


# Rearranging the columns
employee_info = employee_info[['employee_id', 'employee_name', 'employee_position', 'employee_address', 'employee_email', 'employee_phone_number', 'employee_salary']]

# First look
employee_info

### I have successfully created 8 fake tables for the company. Now I will save those 8 tables to drive by csv format.

In [ ]:
tables= ['customer_info', 'product_info', 'order_info', 'order_details', 'payment_details', 'inventory_info', 'supplier_info', 'employee_info']

In [ ]:
for i in tables:
  eval(i).to_csv(f"/content/drive/MyDrive/Raksha pipes/csv/{i}.csv")

In [ ]:
# for i in tables:
#   eval(i).to_excel(f"/content/drive/MyDrive/Raksha pipes/excel/{i}.xlsx")